In [1]:
'''
Experiment Description： 使用MRIbydai_minbox（即加入一定像素的边缘），来进行分类
DATA: train data 共600， 每类150； validation data 共233， 分类为
'''
import tensorflow as tf 
import numpy as np 
import os 

batch_size = 20
learning_rate = 1e-3
class_names = ['Her2', 'luminal_A', 'luminal_B', 'TN']#每个分类的名称

In [4]:
#获取data文件路径
data_dir = 'F:/分子分型/X线MRIbydai_minbox/MRIbydai_minbox/'
Her2_dir = data_dir + 'Her_2/'
luminal_A_dir = data_dir + 'luminal_A/'
luminal_B_dir = data_dir + 'luminal_B/'
TN_dir = data_dir + 'TN/'

In [5]:
#读取每个类别下面图片的数量
Her2_names = [Her2_dir + filename for filename in os.listdir(Her2_dir)]
luminal_A_names = [luminal_A_dir + filename for filename in os.listdir(luminal_A_dir)]
luminal_B_names = [luminal_B_dir + filename for filename in os.listdir(luminal_B_dir)]
TN_names = [TN_dir + filename for filename in os.listdir(TN_dir)]

print(len(Her2_names), len(luminal_A_names), len(luminal_B_names), len(TN_names))

270 179 211 173


In [6]:
#为了保证每类样本的train数量，每类样本训练数量分别提取前150个,共600
Her2_train_names = Her2_names[0:150]
luminal_A_train_names = luminal_A_names[0:150]
luminal_B_train_names = luminal_B_names[0:150]
TN_train_names = TN_names[0:150]

In [7]:
train_names = Her2_train_names + luminal_A_train_names + luminal_B_train_names + TN_train_names
print(len(train_names))

600


In [8]:
#创建train_labels
train_labels = [0] * len(Her2_train_names) + [1] * len(luminal_A_train_names) + [2] * len(luminal_B_train_names) + [3] * len(TN_train_names)
print(len(train_labels))

600


In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_names, train_labels))

In [10]:
def _map(imagename, label):
    image_string = tf.io.read_file(imagename)
    image_decoded = tf.image.decode_png(image_string, channels=1)
    image_resized = tf.image.resize(image_decoded, [40,40])/255.0
    return image_resized, label

In [11]:
train_dataset = train_dataset.map(
    map_func= _map,
    num_parallel_calls= tf.data.experimental.AUTOTUNE
)

In [12]:
train_dataset = train_dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [13]:
#检验数据是否倒入正确
print(train_dataset)

<PrefetchDataset shapes: ((None, 40, 40, 1), (None,)), types: (tf.float32, tf.int32)>


In [14]:
#取余下数据集进行测试
Her2_test_names = Her2_names[150:]
luminal_A_test_names = luminal_A_names[150:]
luminal_B_test_names = luminal_B_names[150:]
TN_test_names = TN_names[150:]
test_names = Her2_test_names + luminal_A_test_names + luminal_B_test_names + TN_test_names
print(len(test_names)) 

233


In [15]:
test_labels = [0] * len(Her2_test_names) + [1] * len(luminal_A_test_names) + [2] * len(luminal_B_test_names) + [3] * len(TN_test_names)
print(len(test_labels))

233


In [16]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_names, test_labels))
test_dataset = test_dataset.map(_map).batch(batch_size)
print(test_dataset)

<BatchDataset shapes: ((None, 40, 40, 1), (None,)), types: (tf.float32, tf.int32)>


In [17]:
#搭建网络
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, 3, activation = tf.nn.relu, input_shape = (40,40,1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, activation = tf.nn.relu),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, activation = tf.nn.relu),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation = tf.nn.relu),
    tf.keras.layers.Dense(64, activation = tf.nn.relu),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation = tf.nn.sigmoid)
])

In [18]:
num_epochs = 40

In [19]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate),
    loss = tf.keras.losses.sparse_categorical_crossentropy,
    metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 38, 38, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 19, 19, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 17, 17, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 128)         147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0

In [21]:
model.fit(train_dataset, validation_data=test_dataset, epochs = num_epochs)

Epoch 1/40
30/30 [==============================] - 1s 40ms/step - loss: 1.3900 - sparse_categorical_accuracy: 0.2317 - val_loss: 1.3905 - val_sparse_categorical_accuracy: 0.1288
Epoch 2/40
30/30 [==============================] - 0s 10ms/step - loss: 1.3852 - sparse_categorical_accuracy: 0.2533 - val_loss: 1.3921 - val_sparse_categorical_accuracy: 0.0815
Epoch 3/40
30/30 [==============================] - 0s 9ms/step - loss: 1.3724 - sparse_categorical_accuracy: 0.3333 - val_loss: 1.3591 - val_sparse_categorical_accuracy: 0.2876
Epoch 4/40
30/30 [==============================] - 0s 9ms/step - loss: 1.3214 - sparse_categorical_accuracy: 0.3883 - val_loss: 1.3244 - val_sparse_categorical_accuracy: 0.3047
Epoch 5/40
30/30 [==============================] - 0s 8ms/step - loss: 1.2655 - sparse_categorical_accuracy: 0.4217 - val_loss: 1.4016 - val_sparse_categorical_accuracy: 0.1974
Epoch 6/40
30/30 [==============================] - 0s 8ms/step - loss: 1.2005 - sparse_categorical_accuracy

In [22]:
model1 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation = tf.nn.relu, input_shape = (40,40,1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 5, activation = tf.nn.relu),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation = tf.nn.relu),
    tf.keras.layers.Dense(4, activation = tf.nn.sigmoid)
])

In [23]:
model1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate),
    loss = tf.keras.losses.sparse_categorical_crossentropy,
    metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)
model1.fit(train_dataset, validation_data=test_dataset, epochs = num_epochs)

Epoch 1/40
30/30 [==============================] - 0s 11ms/step - loss: 1.3776 - sparse_categorical_accuracy: 0.3050 - val_loss: 1.4387 - val_sparse_categorical_accuracy: 0.1159
Epoch 2/40
30/30 [==============================] - 0s 7ms/step - loss: 1.3424 - sparse_categorical_accuracy: 0.3550 - val_loss: 1.3920 - val_sparse_categorical_accuracy: 0.1116
Epoch 3/40
30/30 [==============================] - 0s 7ms/step - loss: 1.2558 - sparse_categorical_accuracy: 0.4300 - val_loss: 1.5045 - val_sparse_categorical_accuracy: 0.2060
Epoch 4/40
30/30 [==============================] - 0s 7ms/step - loss: 1.1395 - sparse_categorical_accuracy: 0.4667 - val_loss: 1.2982 - val_sparse_categorical_accuracy: 0.3734
Epoch 5/40
30/30 [==============================] - 0s 7ms/step - loss: 0.9961 - sparse_categorical_accuracy: 0.5867 - val_loss: 1.5825 - val_sparse_categorical_accuracy: 0.3391
Epoch 6/40
30/30 [==============================] - 0s 8ms/step - loss: 0.9237 - sparse_categorical_accuracy: